In [11]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk
import math
from spyci import spyci

#### Technology data

In [12]:
devices = ['nfet_01v8_lvt', 'nfet_01v8', 'pfet_01v8_lvt', 'pfet_01v8']
n = lk('../../lookuptable/skywater130a/nfet_01v8.mat')
p = lk('../../lookuptable/skywater130a/pfet_01v8.mat')
n_mat = scipy.io.loadmat('../../lookuptable/skywater130a/nfet_01v8.mat', squeeze_me=True)
p_mat = scipy.io.loadmat('../../lookuptable/skywater130a/pfet_01v8.mat', squeeze_me=True)

#### Specifications

In [34]:
A1 = 80 # 10 V/V 
A2 = 400 # 100 V/V 

#### Design choices

In [35]:
gm_id = np.zeros(5) 
gm = np.zeros(5) 
l = np.zeros(5) 
id = np.zeros(5)
gm_id[1] = 20
gm_id[3] = 12
gm_id[2] = 10
gm_id[0] = 10

#### Sizing and benchmarking

In [36]:
# A1 = gm[1]/(gds[1]+gds[2])
i_tail = 2e-6
id[1] = i_tail/2 
l_vec1 = n_mat['nfet_01v8']['L'].flatten()[0]
gm_gds_vec1 = n.lookup('GM_GDS', GM_ID=gm_id[1], L=l_vec1, VDS = 0.3)

# Pick the shortest L that meets gm/gds requirement 
l_index1 = next(x for x, val in enumerate(gm_gds_vec1) if val > A1) 
gm_gds1 = gm_gds_vec1[l_index1]
l[1] = l_vec1[l_index1] 

In [37]:
id[3] = i_tail
l_vec3 = p_mat['pfet_01v8']['L'].flatten()[0] 
gm_gds_vec3 = p.lookup('GM_GDS', GM_ID=gm_id[3], L=l_vec3) 

# Pick the shortest L that meets gm/gds requirement 
l_index3 = next(x for x, val in enumerate(gm_gds_vec3) if val > A2) 
l[3] = l_vec3[l_index3] 
gm[3] = gm_id[3] * id[3] 
rz = 1/gm[3] 

In [38]:
# rz = 589 + 386*l/0.35/m
mrz = 1
lrz = 0.35*mrz*(rz - 589)/386

# cz = m * w * w * 2e-15 + (w + w) * 0.38e-15
cz = 1e-15
mc = 4
def solve_w(mc, cz):
    # constants
    a = 2 * mc * 1e-15
    b = 0.76e-15

    # quadratic formula: w = (-b + sqrt(b^2 + 4ac)) / (2a)
    discriminant = b**2 + 4 * a * cz
    if discriminant < 0:
        raise ValueError("No real solution for given m and cz")

    w_pos = (-b + math.sqrt(discriminant)) / (2 * a)  # positive root
    return w_pos
wc = solve_w(mc,cz)

rs = 1e3
mrs = 1
lrs = 0.35*mrs*(rs - 589)/386

In [39]:
id[2] = id[1]
l[2] = l[1]

id[0] = i_tail
l[0] = l[1]

id[4] = i_tail
l[4] = l[1]

In [40]:
w = np.zeros(5)
# w0 = id[0]/n.lookup('ID_W', GM_ID=range(5,25), L=l[0], VDS = 0.7)
w[0] = id[0]/n.lookup('ID_W', GM_ID=gm_id[0], L=l[0])
w[1] = id[1]/n.lookup('ID_W', GM_ID=gm_id[1], L=l[1]) 
w[2] = id[2]/p.lookup('ID_W', GM_ID=gm_id[2], L=l[2]) 
w[3] = id[3]/p.lookup('ID_W', GM_ID=gm_id[3], L=l[3])
w[4] = id[4]/p.lookup('ID_W', GM_ID=gm_id[4], L=l[4])
nf = [1,1,1,1,1]

In [41]:
gm_cgg=np.zeros(len(l))
gm_cdd=np.zeros(len(l))
gm_cgd=np.zeros(len(l))
gm_gds=np.zeros(len(l))
sth_gm=np.zeros(len(l))

for i in range(len(l)):
    if type[i] == 'n':
      gm_id[i] = n.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgg[i] = n.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cdd[i] = n.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_cgd[i] = n.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      gm_gds[i] = n.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
      sth_gm[i] = n.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i], VDS=0.3)
    else:
      gm_id[i] = p.lookup('GM_ID', ID_W=id[i]/w[i], L=l[i]) 
      gm_cgg[i] = p.lookup('GM_CGG', ID_W=id[i]/w[i], L=l[i])
      gm_cdd[i] = p.lookup('GM_CDD', ID_W=id[i]/w[i], L=l[i])
      gm_cgd[i] = p.lookup('GM_CGD', ID_W=id[i]/w[i], L=l[i])
      gm_gds[i] = p.lookup('GM_GDS', ID_W=id[i]/w[i], L=l[i])
      sth_gm[i] = p.lookup('STH_GM', ID_W=id[i]/w[i], L=l[i])

gm = id*gm_id
gds = gm/gm_gds
cgg = gm/gm_cgg
cdd = gm/gm_cgg
cgd = gm/gm_cgd

df = pd.DataFrame( [gm_id, gm_cgg/1e9/2/np.pi, gm_gds, gm/1e-6, gds/1e-6, cgg/1e-15, cdd/1e-15, cgd/1e-15], \
                   ['gm_id (S/A)', 'ft (GHz)', 'gm_gds', 'gm (uS)', 'gds (uS)', 'cgg (fF)', 'cdd (fF)', 'cgd (fF)'], columns=['M0', 'M1', 'M2', 'M3', 'M4']); df.round(2)

Look up warning: ID_W input larger than maximum! Output is NaN
Look up warning: ID_W input larger than maximum! Output is NaN
Look up warning: ID_W input larger than maximum! Output is NaN
Look up warning: ID_W input larger than maximum! Output is NaN
Look up warning: ID_W input larger than maximum! Output is NaN
Look up warning: ID_W input larger than maximum! Output is NaN


,M0,M1,M2,M3,M4
gm_id (S/A),5.33,14.19,10.00,12.00,2.36
ft (GHz),1.78,0.47,0.95,0.27,2.41
gm_gds,87.07,200.40,157.69,434.72,28.13
gm (uS),10.66,14.19,10.00,24.00,4.72
gds (uS),0.12,0.07,0.06,0.06,0.17
cgg (fF),0.95,4.84,1.68,14.31,0.31
cdd (fF),0.95,4.84,1.68,14.31,0.31
cgd (fF),0.01,0.08,0.03,0.15,0.00


In [42]:
df = pd.DataFrame( [w, l, nf], \
                   ['w (um)', 'l (um)', 'nf',], columns=['M0', 'M1', 'M2', 'M3', 'M4']); df.round(2)

,M0,M1,M2,M3,M4
w (um),0.27,1.67,0.52,2.9,0.03
l (um),0.60,0.60,0.60,1.0,0.60
nf,1.00,1.00,1.00,1.0,1.00


#### Write spice include file

In [43]:
# override with baseline values if desired
with open('../xschem/sizing_opamp.spice', 'w') as file:
    file.write(".param i_tail = " + "{:.2e}".format(i_tail) + '\n')
    file.write(".param lrz = " + "{:.2e}".format(lrz) + '\n')
    file.write(".param mrz = " + "{:.2e}".format(mrz) + '\n')
    file.write(".param lrs = " + "{:.2e}".format(lrs) + '\n')
    file.write(".param mrs = " + "{:.2e}".format(mrs) + '\n')
    file.write(".param mc = " + "{:.2e}".format(mc) + '\n')
    file.write(".param wc = " + "{:.2e}".format(wc) + '\n')

    for i in range(len(l)):
      file.write(".param w%d = " % i + "{:.2f}".format(w[i]) + '\n')
      file.write(".param l%d = " % i + "{:.2f}".format(l[i]) + '\n')
      file.write(".param nf%d = " % i + "{:.2f}".format(nf[i]) + '\n')